In [ ]:
import pandas as pd
import numpy as np
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from nltk.sentiment import SentimentIntensityAnalyzer
import praw
import time

In [ ]:
sia = SentimentIntensityAnalyzer()

In [ ]:
reddit = praw.Reddit(client_id='rLAtyw4_uLGI-Zpd1WdxwA', client_secret='tXotg2-KyG9YL4Q_iIA5c5ZRu68dxw', user_agent='SentiAnalysisBot/1.0')

In [ ]:
def get_sentiment_score(ticker):
    subreddit = reddit.subreddit("stocks")
    mentions = subreddit.search(ticker, limit=1)
    sentiment_scores = [sia.polarity_scores(post.title)['compound'] for post in mentions]
    time.sleep(1)
    return np.mean(sentiment_scores) if sentiment_scores else 0

In [ ]:
def get_simple_sentiment_score(ticker):
    subreddit = reddit.subreddit("stocks")
    posts = list(subreddit.hot(limit=20))  # Just top 100 posts
    mentions = [post.title for post in posts if ticker in post.title]

    if not mentions:
        return 0  # No mention, neutral

    sentiment_scores = [sia.polarity_scores(title)['compound'] for title in mentions]
    return np.mean(sentiment_scores)


In [ ]:
#df = pd.DataFrame(stock_data)
df = pd.read_csv('constituents-financials.csv')

In [ ]:
df = df.drop(labels=['SEC Filings','Sector','52 Week High','52 Week Low'],axis=1)

In [ ]:
df = df.dropna()

In [ ]:
#df['Sentiment'] = df['Symbol'].apply(get_simple_sentiment_score)

In [ ]:
df.value_counts()

In [ ]:
df.head(10)

In [ ]:
df['good_pe'] = df['Price/Earnings'].between(0, 25)  # PE ratio between 0 and 25 is "good"
df['good_eps'] = df['Earnings/Share'] >= 0                 # EPS greater than or equal to 0 is "good"
#df['good_roe'] = df['roe'].between(0.15, 0.20)   # ROE between 0.10 and 0.25 is "good"
#df['good_debt_equity'] = df['debt_to_equity'].between(1.5,2) 
df['good_mc'] = df['Market Cap'] >= 2e9         # Market Cap greater than or equal to 1 billion is "good"
df['good_pb'] = df['Price/Book'].between(0, 3)
#df['good_current'] = df['current_ratio'].between(1.5,3)
#df['good_sentiment'] = df['sentiment'] > 0 
df['good_div'] = df['Dividend Yield'].between(0.02, 0.06)

In [ ]:
df['good_stock'] = df[['good_pe', 'good_eps', 'good_mc', 'good_pb', 'good_div']].all(axis=1).astype(int)

In [ ]:
df

In [ ]:
scaler = StandardScaler()

In [ ]:
features = ['Price/Earnings','Earnings/Share','Market Cap','Price/Book','Dividend Yield']
X = df[features]

In [ ]:
X_scaled = scaler.fit_transform(X)

In [ ]:
clf = LogisticRegression()

# Train the model
clf.fit(X_scaled, df['good_stock'])

# Predict the "good stock" for each stock in the dataset
df['predicted_good_stock'] = clf.predict(X_scaled)

In [ ]:
#see output
df

In [ ]:
#check the model's metrics
print("Accuracy",accuracy_score(df['good_stock'],df['predicted_good_stock']))
print("Recall:",recall_score(df['good_stock'],df['predicted_good_stock']))

In [ ]:
joblib.dump(clf, 'logistic_model.pkl')

In [ ]:
df.to_csv('predicted_stocks.csv',index=False)